# World Happiness & Economic and social indicators correlations

## Data Collection

In [283]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings

# Version Check
print(f"pandas: {pd.__version__}, numpy: {np.__version__}, sklearn: {sklearn.__version__}, seaborn: {sns.__version__}")

# Warnings Off
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load Datasets
def load_dataset(path):
    """Load CSV dataset with standard parameters."""
    return pd.read_csv(path, encoding='utf-8-sig', skip_blank_lines=True)

happiness_df = load_dataset('../data/raw/World_Happiness_Index_2015_2024.csv')
gdp_df = load_dataset("../data/raw/GDP_per_capita_2015_2023.csv")
edu_df = load_dataset("../data/raw/Education_Attainment_Bachelor_2015_2023.csv")
life_exp_wb = load_dataset("../data/raw/WorldBank_Life_Expectancy_2015_2022.csv")
life_exp_2023 = load_dataset("../data/raw/OECD_Life_Expectancy_Total_2023_Cleaned.csv")
unemp_df = load_dataset("../data/raw/Unemployment_Rate_2015_2023.csv")

pandas: 2.2.3, numpy: 2.2.2, sklearn: 1.6.1, seaborn: 0.13.2


## Initial Inspection

In [284]:
# Inspect data
happiness_df.head()
# gdp_df.head()
# edu_df.head()
# life_exp_wb.head()
# life_exp_2023.head()
# unemp_df.head()

,Year,Country,Happiness Score
0,2024,Finland,7.736
1,2023,Afghanistan,1.721
2,2022,Afghanistan,1.859
3,2021,Afghanistan,2.404
4,2020,Afghanistan,2.523


In [285]:
happiness_df['Year'].value_counts().sort_index()

Year
2015    157
2016    155
2017    156
2018    156
2019    153
2020    149
2021    146
2022    137
2023    143
2024    147
Name: count, dtype: int64

In [286]:
gdp_df['Country Name'].nunique()

260

In [288]:
happiness_df.info()
gdp_df.info()
edu_df.info()
life_exp_wb.info()
life_exp_2023.info()
unemp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             1499 non-null   int64  
 1   Country          1499 non-null   object 
 2   Happiness Score  1499 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 35.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  260 non-null    object 
 1   Country Code  260 non-null    object 
 2   2015          259 non-null    float64
 3   2016          258 non-null    float64
 4   2017          258 non-null    float64
 5   2018          258 non-null    float64
 6   2019          259 non-null    float64
 7   2020          258 non-null    float64
 8   2021          258 non-null    float64
 9   2022          255 

## Initial Data Cleaning & Preprocessing

### Data Reshaping

In [289]:
# Convert Wide to Long format
def wide_to_long(df, value_name):
    """Convert wide format to long format."""
    
    # Standardise column names
    if 'Country Name' in df.columns:
        df = df.rename(columns={'Country Name': 'Country'})
    
    # Exclude 'Country Code' if present
    id_vars = ['Country']
    if 'Country Code' in df.columns:
        id_vars.append('Country Code')
    
    df_long = df.melt(id_vars=id_vars, var_name='Year', value_name=value_name)
    
    # Drop 'Country Code' after melt
    if 'Country Code' in df_long.columns:
        df_long = df_long.drop(columns=['Country Code'])
    
    # Convert year to int (skip invalid rows safely)
    df_long = df_long[df_long['Year'].str.isnumeric()]
    df_long['Year'] = df_long['Year'].astype(int)
    
    return df_long

# Apply conversion
gdp_long = wide_to_long(gdp_df, 'GDP_per_capita')
edu_long = wide_to_long(edu_df, 'Education_Attainment')
life_exp_wb_long = wide_to_long(life_exp_wb, 'Life_Expectancy')
unemp_long = wide_to_long(unemp_df, 'Unemployment_Rate')

# OECD life expectancy is already long format, just rename columns
life_exp_oecd_long = life_exp_2023.rename(columns={'Life Expectancy 2023': 'Life_Expectancy', 'Country': 'Country'})
life_exp_oecd_long['Year'] = 2023

### Section Subtitle 

In [290]:
# Define country name standardisation function
def standardise_country_column(df):
    """
    Standardise 'Country' column: lowercasing and stripping whitespace.
    """
    df['Country'] = df['Country'].str.strip().str.lower()
    return df

# Apply to all long datasets
for df in [happiness_df, gdp_long, edu_long, life_exp_wb_long, life_exp_oecd_long, unemp_long]:
    standardise_country_column(df)

###  Removing Non-Country Entries for Dataset Integrity

In [291]:
import re

# Define Function to Identify Group/Region Names
def is_group(name):
    patterns = [
        'income', 'region', 'area', 'ida', 'ibrd', 'small states', 'world', 'union',
        'total', 'blend', 'dividend', 'members', 'arab world', 'latin america',
        'sub-saharan', 'europe & central asia', 'pacific island', 'fragile', 'oecd',
        'post-demographic', 'pre-demographic', 'middle east', 'caribbean', 'heavily indebted'
    ]
    for p in patterns:
        if re.search(p, str(name).lower()):
            return True
    return False

# Standardised Filtering Function
def filter_out_groups(df, df_name):
    before = df['Country'].nunique()
    df = df[~df['Country'].apply(is_group)]
    after = df['Country'].nunique()
    print(f"Removed non-country entries: {before - after} entries removed from {df_name}. Remaining: {after} countries.")
    return df

# Apply to all datasets
gdp_long = filter_out_groups(gdp_long, 'GDP')
edu_long = filter_out_groups(edu_long, 'Education')
life_exp_wb_long = filter_out_groups(life_exp_wb_long, 'Life Exp WB')
life_exp_oecd_long = filter_out_groups(life_exp_oecd_long, 'Life Exp OECD')
unemp_long = filter_out_groups(unemp_long, 'Unemployment')

Removed non-country entries: 42 entries removed from GDP. Remaining: 218 countries.
Removed non-country entries: 1 entries removed from Education. Remaining: 179 countries.
Removed non-country entries: 42 entries removed from Life Exp WB. Remaining: 217 countries.
Removed non-country entries: 0 entries removed from Life Exp OECD. Remaining: 28 countries.
Removed non-country entries: 42 entries removed from Unemployment. Remaining: 193 countries.


### Checking Country Mismatches Against 2024 Target Set

In [292]:
# Extract 2024 Test Set Countries
target_countries = set(happiness_df[happiness_df['Year'] == 2024]['Country'].unique())

# Identify unmatched countries in each dataset
def check_country_mismatch(df, name):
    mismatch = set(df['Country'].unique()) - target_countries
    print(f"{name} mismatched countries ({len(mismatch)}): {sorted(mismatch)}\n")

# Apply check to long format datasets
check_country_mismatch(gdp_long, 'GDP')
check_country_mismatch(edu_long, 'Education')
check_country_mismatch(life_exp_wb_long, 'Life Exp WB')
check_country_mismatch(life_exp_oecd_long, 'Life Exp OECD')
check_country_mismatch(unemp_long, 'Unemployment')

GDP mismatched countries (88): ['africa eastern and southern', 'africa western and central', 'american samoa', 'andorra', 'angola', 'antigua and barbuda', 'aruba', 'bahamas, the', 'barbados', 'belarus', 'bermuda', 'bhutan', 'brunei darussalam', 'burundi', 'cabo verde', 'cayman islands', 'central african republic', 'central europe and the baltics', 'channel islands', 'congo, dem. rep.', 'congo, rep.', "cote d'ivoire", 'cuba', 'curacao', 'djibouti', 'dominica', 'east asia & pacific', 'egypt, arab rep.', 'equatorial guinea', 'faroe islands', 'fiji', 'french polynesia', 'gambia, the', 'greenland', 'grenada', 'guam', 'guinea-bissau', 'guyana', 'haiti', 'hong kong sar, china', 'iran, islamic rep.', 'isle of man', 'kiribati', 'korea, rep.', 'kyrgyz republic', 'least developed countries: un classification', 'liechtenstein', 'macao sar, china', 'maldives', 'marshall islands', 'micronesia, fed. sts.', 'moldova', 'monaco', 'nauru', 'new caledonia', 'north america', 'northern mariana islands', 'pa

In [293]:
# Extract Country sets from each dataset
gdp_countries = set(gdp_long['Country'].unique())
edu_countries = set(edu_long['Country'].unique())
life_exp_wb_countries = set(life_exp_wb_long['Country'].unique())
life_exp_oecd_countries = set(life_exp_oecd_long['Country'].unique())
unemp_countries = set(unemp_long['Country'].unique())

# Identify countries present in Happiness 2024 but missing in Unemployment dataset
missing_countries = target_countries - unemp_countries
print(f"Countries in Happiness 2024 but missing in Unemployment Dataset: {len(missing_countries)}")
print(sorted(missing_countries))

Countries in Happiness 2024 but missing in Unemployment Dataset: 18
['congo', 'côte d’ivoire', 'dr congo', 'egypt', 'gambia', 'hong kong sar of china', 'iran', 'kosovo', 'kyrgyzstan', 'republic of korea', 'republic of moldova', 'slovakia', 'state of palestine', 'taiwan province of china', 'trinidad and tobago', 'türkiye', 'venezuela', 'yemen']


In [294]:
from fuzzywuzzy import process

print("\nPossible Matches in Unemployment Dataset:")
for country in missing_countries:
    match = process.extractOne(country, list(unemp_countries))
    print(f"{country} ↔ Closest Match: {match}")


Possible Matches in Unemployment Dataset:
gambia ↔ Closest Match: ('gambia, the', 90)
republic of korea ↔ Closest Match: ('korea, rep.', 86)
trinidad and tobago ↔ Closest Match: ('st. vincent and the grenadines', 86)
taiwan province of china ↔ Closest Match: ('china', 90)
egypt ↔ Closest Match: ('egypt, arab rep.', 90)
türkiye ↔ Closest Match: ('turkiye', 92)
yemen ↔ Closest Match: ('yemen, rep.', 90)
republic of moldova ↔ Closest Match: ('moldova', 90)
kyrgyzstan ↔ Closest Match: ('kazakhstan', 60)
slovakia ↔ Closest Match: ('slovenia', 75)
kosovo ↔ Closest Match: ('lesotho', 46)
iran ↔ Closest Match: ('iran, islamic rep.', 90)
venezuela ↔ Closest Match: ('venezuela, rb', 95)
côte d’ivoire ↔ Closest Match: ("cote d'ivoire", 96)
hong kong sar of china ↔ Closest Match: ('hong kong sar, china', 95)
congo ↔ Closest Match: ('congo, rep.', 90)
state of palestine ↔ Closest Match: ('eswatini', 64)
dr congo ↔ Closest Match: ('congo, dem. rep.', 86)


### Standardising Country Names Across Datasets for Reliable Merging

In [295]:
country_mapping = {
    'yemen': 'yemen, rep.',
    'syria': 'syrian arab republic',
    'republic of moldova': 'moldova',
    'egypt': 'egypt, arab rep.',
    'hong kong sar of china': 'hong kong sar, china',
    'macedonia': 'north macedonia',
    'congo': 'congo, dem. rep.',
    'iran': 'iran, islamic rep.',
    'türkiye': 'turkiye',
    'gambia': 'gambia, the',
    'dr congo': 'congo, dem. rep.',
    'côte d’ivoire': "cote d'ivoire",
    'north cyprus': 'cyprus',
    'republic of korea': 'korea, rep.',
    'venezuela': 'venezuela, rb',
    'swaziland': 'eswatini',
    'kyrgyzstan': 'kyrgyz republic',
    'slovakia': 'slovak republic',
    'taiwan province of china': 'taiwan',
}

# Apply mapping to long format datasets
for df in [happiness_df, gdp_long, edu_long, life_exp_wb_long, life_exp_oecd_long, unemp_long]:
    df['Country'] = df['Country'].replace(country_mapping)

In [296]:
# Define function to check mismatch
def check_mismatch(df, name):
    df_countries = set(df['Country'].unique())
    missing = target_countries - df_countries
    print(f"[{name}] Missing countries after mapping: {len(missing)}")
    print(sorted(missing))

# Check for all datasets
check_mismatch(gdp_long, 'GDP')
check_mismatch(edu_long, 'Education')
check_mismatch(life_exp_wb_long, 'Life Exp WB')
check_mismatch(life_exp_oecd_long, 'Life Exp OECD')
check_mismatch(unemp_long, 'Unemployment')

[GDP] Missing countries after mapping: 17
['congo', 'côte d’ivoire', 'dr congo', 'egypt', 'gambia', 'hong kong sar of china', 'iran', 'kyrgyzstan', 'republic of korea', 'republic of moldova', 'slovakia', 'state of palestine', 'taiwan province of china', 'trinidad and tobago', 'türkiye', 'venezuela', 'yemen']
[Education] Missing countries after mapping: 24
['argentina', 'congo', 'côte d’ivoire', 'dr congo', 'egypt', 'gabon', 'gambia', 'hong kong sar of china', 'iran', 'kosovo', 'kyrgyzstan', 'libya', 'morocco', 'peru', 'republic of korea', 'republic of moldova', 'slovakia', 'state of palestine', 'taiwan province of china', 'trinidad and tobago', 'türkiye', 'ukraine', 'venezuela', 'yemen']
[Life Exp WB] Missing countries after mapping: 17
['congo', 'côte d’ivoire', 'dr congo', 'egypt', 'gambia', 'hong kong sar of china', 'iran', 'kyrgyzstan', 'republic of korea', 'republic of moldova', 'slovakia', 'state of palestine', 'taiwan province of china', 'trinidad and tobago', 'türkiye', 'venezu

### Country Alignment & Filtering Across Datasets

In [297]:
# Store datasets
datasets = {
    'GDP': gdp_long,
    'Education': edu_long,
    'Life Exp WB': life_exp_wb_long,
    'Life Exp OECD': life_exp_oecd_long,
    'Unemployment': unemp_long
}

print(f"📊 Base: Total Happiness countries (2024): {len(target_countries)}")

# Loop through datasets to calculate and display common and difference counts
for name, df in datasets.items():
    df_countries = set(df['Country'].unique())
    common_countries = df_countries & target_countries
    difference_countries = df_countries - target_countries
    
    print(f"\n🔎 {name} Dataset:")
    print(f"- Total countries in dataset: {len(df_countries)}")
    print(f"- Common countries with Happiness 2024: {len(common_countries)}")
    print(f"- Countries in {name} but not in Happiness 2024: {len(difference_countries)}")
    
    # Filter the dataset to include only common countries
    df_filtered = df[df['Country'].isin(target_countries)]
    
    # Update the dataset with the filtered data
    datasets[name] = df_filtered

📊 Base: Total Happiness countries (2024): 147

🔎 GDP Dataset:
- Total countries in dataset: 218
- Common countries with Happiness 2024: 130
- Countries in GDP but not in Happiness 2024: 88

🔎 Education Dataset:
- Total countries in dataset: 179
- Common countries with Happiness 2024: 123
- Countries in Education but not in Happiness 2024: 56

🔎 Life Exp WB Dataset:
- Total countries in dataset: 217
- Common countries with Happiness 2024: 130
- Countries in Life Exp WB but not in Happiness 2024: 87

🔎 Life Exp OECD Dataset:
- Total countries in dataset: 28
- Common countries with Happiness 2024: 27
- Countries in Life Exp OECD but not in Happiness 2024: 1

🔎 Unemployment Dataset:
- Total countries in dataset: 193
- Common countries with Happiness 2024: 129
- Countries in Unemployment but not in Happiness 2024: 64


## Data Merging & Saving

In [298]:
import os

# Create output directory if not exists
output_dir = '../data/processed/long'
os.makedirs(output_dir, exist_ok=True)

# Merge the life expectancy datasets (World Bank and OECD) on Country and Year
life_exp_combined = pd.merge(
    life_exp_wb_long[['Country', 'Year', 'Life_Expectancy']],
    life_exp_oecd_long[['Country', 'Year', 'Life_Expectancy']],
    on=['Country', 'Year'],
    how='outer',
    suffixes=('_WB', '_OECD')
)

# Combine values
life_exp_combined['Life_Expectancy'] = life_exp_combined['Life_Expectancy_WB'].fillna(
    life_exp_combined['Life_Expectancy_OECD']
)
life_exp_combined.drop(columns=['Life_Expectancy_WB', 'Life_Expectancy_OECD'], inplace=True)

# Save cleaned long datasets
gdp_long.to_csv(f'{output_dir}/gdp_long.csv', index=False)
edu_long.to_csv(f'{output_dir}/edu_long.csv', index=False)
unemp_long.to_csv(f'{output_dir}/unemp_long.csv', index=False)
happiness_df.to_csv(f'{output_dir}/happiness_long.csv', index=False)
life_exp_combined.to_csv(f'{output_dir}/life_exp_long.csv', index=False)
print("✅ All datasets saved successfully!")

✅ All datasets saved successfully!


In [299]:
# Merge datasets step by step
merged_df = happiness_df.copy()

# Sequentially merge
merged_df = pd.merge(merged_df, gdp_long, on=['Country', 'Year'], how='inner')
merged_df = pd.merge(merged_df, edu_long, on=['Country', 'Year'], how='inner')
merged_df = pd.merge(merged_df, life_exp_combined, on=['Country', 'Year'], how='inner')
merged_df = pd.merge(merged_df, unemp_long, on=['Country', 'Year'], how='inner')

# Handle duplicates
duplicates = merged_df[merged_df.duplicated(subset=['Country', 'Year'], keep=False)]
if not duplicates.empty:
    print(f"⚠️ Found {len(duplicates)} duplicate rows. Aggregating with mean...")
    merged_df = merged_df.groupby(['Country', 'Year'], as_index=False).mean(numeric_only=True)
else:
    print("✅ No duplicate rows found.")
    
print(f"Merged dataset shape: {merged_df.shape}")

⚠️ Found 28 duplicate rows. Aggregating with mean...
Merged dataset shape: (1217, 7)


In [300]:
# Filter for training set (exclude 2024)
merged_df = merged_df[merged_df['Year'] < 2024].reset_index(drop=True)

# Extract 2024 target dataset
target_df = happiness_df[happiness_df['Year'] == 2024][['Year', 'Country', 'Happiness Score']]

# Move label to last column
merged_df = merged_df[[col for col in merged_df.columns if col != 'Happiness Score'] + ['Happiness Score']]
target_df = target_df[[col for col in target_df.columns if col != 'Happiness Score'] + ['Happiness Score']]

# Save datasets
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

merged_df.to_csv(f'{output_dir}/merged_dataset.csv', index=False)
target_df.to_csv(f'{output_dir}/target_dataset.csv', index=False)
print("✅ Train and Test datasets saved!")

✅ Train and Test datasets saved!


### Chat GPT Data Cleaning

This notebook summarises the data cleaning process applied to the original dataset. The goal was to prepare the data for analysis and modelling by handling missing values and ensuring consistency across key columns.

---

## ✅ Column-Wise Cleaning Summary

### 1. `Country`
- **Transformation:** Converted all country names to Title Case.
- **Reason:** For consistent formatting and better readability in visualisations and grouping operations.

### 2. `Education_Attainment`
- **Issue:** 462 missing values.
- **Step 1:** Filled missing values using the **median within each country**.
- **Step 2:** For countries where all values were missing (9 records), filled using the **overall median** of the dataset.
- **Reason:** Median is robust to outliers and reflects typical values without being skewed. Country-specific median preserves local distribution.

### 3. `Life_Expectancy`
- **Issue:** 103 missing values.
- **Step 1:** Applied **linear interpolation grouped by country** to fill time-series gaps.
- **Step 2:** Remaining 101 missing values (from countries with no valid entries) were filled using the **global mean**.
- **Reason:** Interpolation maintains temporal trends. For completely missing cases, the mean provides a stable fallback.

### 4. `GDP_per_capita`, `Unemployment_Rate`, `Happiness Score`, `Year`
- **Action:** No missing values; retained as-is.

---

## 🗂️ Output
The cleaned dataset has been saved as:

📁 `gpt_merged_dataset.csv`

All columns are now fully complete with **no missing values** and ready for analysis or modelling.

In [279]:
merged = load_dataset('../data/processed/gpt_merged_dataset.csv')

# Split train, validation, test based on Year
train = merged[merged['Year'] <= 2022]
test  = merged[merged['Year'] == 2023]

train.to_csv(f'../data/processed/gpt_train_dataset.csv', index=False)
test.to_csv(f'../data/processed/gpt_test_dataset(2023).csv', index=False)

## Post-EDA Cleaning 

### 1. time series-based interpolation

In [301]:
# Load the merged dataset
merged = pd.read_csv('../data/processed/merged_dataset_v1.1.csv')

# Perform time series interpolation for each country
for feature in ['Education_Attainment', 'Life_Expectancy']:
    merged[feature] = (
        merged.groupby('Country')[feature]
        .transform(lambda x: x.interpolate(method='linear', limit_direction='both'))
    )

# Save the interpolated version
output_path = '../data/processed/merged_interpolated.csv'
merged.to_csv(output_path, index=False)

print(f"✅ Interpolated dataset saved to: {output_path}")

✅ Interpolated dataset saved to: ../data/processed/merged_interpolated.csv


#### 1.1. Split train/test data

In [303]:
interpolated = pd.read_csv('../data/processed/merged_interpolated.csv')

# Split train, validation, test based on Year
train = interpolated[interpolated['Year'] <= 2022]
test  = interpolated[interpolated['Year'] == 2023]

train.to_csv(f'../data/processed/train_dataset.csv', index=False)
test.to_csv(f'../data/processed/test_dataset(2023).csv', index=False)